In [2]:
!wget https://archive.ics.uci.edu/static/public/222/bank+marketing.zip

--2024-10-11 18:50:24--  https://archive.ics.uci.edu/static/public/222/bank+marketing.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘bank+marketing.zip’

bank+marketing.zip      [         <=>        ] 999.85K   469KB/s    in 2.1s    

2024-10-11 18:50:27 (469 KB/s) - ‘bank+marketing.zip’ saved [1023843]



In [3]:
import pandas as pd

In [186]:
df = pd.read_csv('bank+marketing/bank/bank-full.csv', sep=';')

In [187]:
using_columns = ['age', 'job', 'marital', 'education', 'balance', 'housing', 'contact', 'day', 'month', 'duration', 'campaign', 'pdays', 'previous', 'poutcome', 'y']

In [188]:
df = df[using_columns]

In [189]:
df.head()

,age,job,marital,education,balance,housing,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,58,management,married,tertiary,2143,yes,unknown,5,may,261,1,-1,0,unknown,no
1,44,technician,single,secondary,29,yes,unknown,5,may,151,1,-1,0,unknown,no
2,33,entrepreneur,married,secondary,2,yes,unknown,5,may,76,1,-1,0,unknown,no
3,47,blue-collar,married,unknown,1506,yes,unknown,5,may,92,1,-1,0,unknown,no
4,33,unknown,single,unknown,1,no,unknown,5,may,198,1,-1,0,unknown,no


In [190]:
df.isnull().sum()

age          0
job          0
marital      0
education    0
balance      0
housing      0
contact      0
day          0
month        0
duration     0
campaign     0
pdays        0
previous     0
poutcome     0
y            0
dtype: int64

In [191]:
df.education.value_counts()

education
secondary    23202
tertiary     13301
primary       6851
unknown       1857
Name: count, dtype: int64

In [182]:
df.education.mode()

0    secondary
Name: education, dtype: object

In [192]:
df.dtypes

age           int64
job          object
marital      object
education    object
balance       int64
housing      object
contact      object
day           int64
month        object
duration      int64
campaign      int64
pdays         int64
previous      int64
poutcome     object
y            object
dtype: object

In [193]:
num_columns = ['age', 'balance', 'day', 'duration', 'campaign', 'pdays', 'previous']
cat_columns = ['job', 'marital', 'education', 'housing', 'contact', 'month', 'poutcome']

In [194]:
df[num_columns].corr()

,age,balance,day,duration,campaign,pdays,previous
age,1.000000,0.097783,-0.009120,-0.004648,0.004760,-0.023758,0.001288
balance,0.097783,1.000000,0.004503,0.021560,-0.014578,0.003435,0.016674
day,-0.009120,0.004503,1.000000,-0.030206,0.162490,-0.093044,-0.051710
duration,-0.004648,0.021560,-0.030206,1.000000,-0.084570,-0.001565,0.001203
campaign,0.004760,-0.014578,0.162490,-0.084570,1.000000,-0.088628,-0.032855
pdays,-0.023758,0.003435,-0.093044,-0.001565,-0.088628,1.000000,0.454820
previous,0.001288,0.016674,-0.051710,0.001203,-0.032855,0.454820,1.000000


In [195]:
df.y.value_counts()

y
no     39922
yes     5289
Name: count, dtype: int64

In [196]:
df.y = (df.y == 'yes').astype(int)

In [197]:
df.head()
df.y.value_counts()

y
0    39922
1     5289
Name: count, dtype: int64

In [198]:
from sklearn.model_selection import train_test_split

In [199]:
len(df)

45211

In [200]:
df_full_train, df_test = train_test_split(df, test_size = 0.2, random_state = 42)

In [201]:
df_train, df_val = train_test_split(df_full_train, test_size = 0.25, random_state = 42)

In [202]:
len(df_train), len(df_val), len(df_test)

(27126, 9042, 9043)

In [203]:
df_train = df_train.reset_index(drop = True)
df_val = df_val.reset_index(drop = True)
df_test = df_test.reset_index(drop = True)

In [204]:
y_train = df_train.y.values
y_val = df_val.y.values
y_test = df_test.y.values

del df_train['y']
del df_val['y']
del df_test['y']

In [120]:
from sklearn.metrics import mutual_info_score

In [121]:
df_train.head()

,age,job,marital,education,balance,housing,contact,day,month,duration,campaign,pdays,previous,poutcome
0,32,technician,single,tertiary,1100,yes,cellular,11,aug,67,1,-1,0,unknown
1,38,entrepreneur,married,secondary,0,yes,cellular,17,nov,258,1,-1,0,unknown
2,49,blue-collar,married,secondary,3309,yes,cellular,15,may,349,2,-1,0,unknown
3,37,housemaid,married,primary,2410,no,cellular,4,aug,315,1,-1,0,unknown
4,31,self-employed,married,tertiary,3220,no,cellular,26,aug,74,4,-1,0,unknown


In [122]:
for col in cat_columns:
    print(col, round(mutual_info_score(y_train, df_train[col]), 2))

job 0.01
marital 0.0
education 0.0
housing 0.01
contact 0.01
month 0.03
poutcome 0.03


In [123]:
from sklearn.feature_extraction import DictVectorizer

In [205]:
dv = DictVectorizer(sparse = False)

In [206]:
X_train = dv.fit_transform(df_train.to_dict(orient = 'records'))

In [207]:
X_val = dv.transform(df_val.to_dict(orient = 'records'))

In [208]:
from sklearn.linear_model import LogisticRegression

In [128]:
model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

In [129]:
model.fit(X_train, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [131]:
model.intercept_[0]

np.float64(-0.9846697750691178)

In [135]:
model.coef_[0].round(3)

array([-0.000e+00,  0.000e+00, -7.900e-02,  2.590e-01,  7.600e-02,
       -1.319e+00,  9.000e-03,  4.000e-03, -4.410e-01, -2.450e-01,
       -6.900e-02, -2.300e-01, -1.430e-01, -8.420e-01,  8.500e-02,
       -2.240e-01, -2.660e-01, -3.320e-01, -8.900e-02,  2.810e-01,
       -3.000e-01, -1.320e-01,  2.850e-01, -1.500e-01,  3.200e-02,
       -1.730e-01, -3.350e-01, -4.740e-01, -1.750e-01,  3.000e-03,
       -7.250e-01,  3.880e-01, -3.350e-01, -1.171e+00, -1.036e+00,
        3.130e-01,  1.463e+00, -5.020e-01, -9.760e-01,  7.940e-01,
        7.980e-01, -1.000e-03, -8.090e-01, -6.280e-01,  1.486e+00,
       -1.034e+00,  7.000e-03])

In [144]:
y_val_pred = model.predict_proba(X_val)[:, 1]

In [145]:
churn_decision = (y_val_pred > 0.5)

In [149]:
(y_val == churn_decision).mean()

np.float64(0.9007962840079629)

In [151]:
small_set = ['age', 'balance', 'marital', 'previous']
df_train_small = df_train[small_set]
df_val_small = df_val[small_set]

In [153]:
dvs = DictVectorizer(sparse = False)

In [156]:
X_train_small = dv.fit_transform(df_train_small.to_dict(orient = 'records'))
X_val_small = dv.transform(df_val_small.to_dict(orient = 'records')) 

In [157]:
s_model = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

In [158]:
s_model.fit(X_train_small, y_train)

LogisticRegression(max_iter=1000, random_state=42, solver='liblinear')

In [159]:
y_val_pred_small = s_model.predict_proba(X_val_small)[:, 1]

In [160]:
cd_s = y_val_pred_small > 0.5

In [164]:
base_line = (y_val == cd_s).mean()
base_line

np.float64(0.880336208803362)

In [175]:
for drop in small_set:
    # print(drop)
    df_train_small_drop = df_train[small_set]
    df_val_small_drop = df_val[small_set]
    del df_train_small_drop[drop]
    del df_val_small_drop[drop]
    # print(df_train_small_drop.columns)
    # print(df_val_small_drop.columns)

    s_model_drop = LogisticRegression(solver='liblinear', C=1.0, max_iter=1000, random_state=42)

    dv_drop = DictVectorizer(sparse = False)

    X_train_small_drop = dv_drop.fit_transform(df_train_small_drop.to_dict(orient = 'records'))
    X_val_small_drop = dv_drop.transform(df_val_small_drop.to_dict(orient = 'records')) 

    s_model_drop.fit(X_train_small_drop, y_train)

    y_val_pred_small_drop = s_model_drop.predict_proba(X_val_small_drop)[:, 1]
    cd_drop = (y_val_pred_small_drop > 0.5)
    drop_ac = (y_val == cd_drop).mean()
    print(drop, base_line - drop_ac)

age -0.0001105950011059953
balance 0.0
marital 0.00011059500110588427
previous -0.0013271400132713884


In [211]:
for c in [0.01, 0.1, 1, 10, 100]:
    model_c = LogisticRegression(solver='liblinear', C=c, max_iter=1000, random_state=42)
    model_c.fit(X_train, y_train)

    y_val_c = model_c.predict_proba(X_val)[:, 1]
    cd_c = (y_val_c > 0.5)
    c_ac = (y_val == cd_c).mean()
    print(c, c_ac)

0.01 0.8976996239769962
0.1 0.9007962840079629
1 0.9007962840079629
10 0.9012386640123866
100 0.9010174740101747
